In [6]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.xgboost.model import XGBoostModel

session = sagemaker.Session()
role = get_execution_role()

xgb_model = XGBoostModel(
    model_data='s3://transactly-data-lake/Fraud_data/raw/output/models/xgb/pipelines-59g49bgqtquu-TrainXGBoost-IBVJnoJB52/output/model.tar.gz',  # Replace with your actual path
    role=role,
    entry_point='xgb_inference.py',  # Optional
    framework_version='1.7-1',
    sagemaker_session=session
)

xgb_predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='xgb-fraud-endpoint1'
)


------!

In [1]:
import boto3

sm_client = boto3.client('sagemaker')

response = sm_client.list_endpoints()
for ep in response['Endpoints']:
    print(ep['EndpointName'], ep['EndpointStatus'])


fraud-detector-endpoint InService


In [19]:
import boto3
import json

runtime = boto3.client("sagemaker-runtime")

response = runtime.invoke_endpoint(
    EndpointName="xgb-fraud-endpoint1",
    ContentType="application/json",
    Body=json.dumps({
        "amount": 500.99,
        "ip_risk_score": 0.17,
        "country_code": 12,                 
        "device_age_days": 18,
        "hour_of_day": 417,
        "is_vpn": 0,
        "distance_from_home_km": 15.4,
        "category_id": 37,                  
        "cvv_attempts": 0,
        "session_duration_sec": 2125,
        "is_new_device": 1,
        "payment_method": 2,               
        "shipping_billing_match": 1
    })
)

result = response["Body"].read().decode("utf-8")
print(result)  # → something like: [0.842]


[0.17367036640644073]


In [20]:
import boto3
import json

runtime = boto3.client("sagemaker-runtime")

response = runtime.invoke_endpoint(
    EndpointName="xgb-fraud-endpoint1",
    ContentType="application/json",
    Body=json.dumps({
        "amount": 500000009.99,
        "ip_risk_score": 0.87,
        "country_code": 12,                 
        "device_age_days": 1810,
        "hour_of_day": 417,
        "is_vpn": 0,
        "distance_from_home_km": 15.4,
        "category_id": 37,                  
        "cvv_attempts": 0,
        "session_duration_sec": 2125,
        "is_new_device": 1,
        "payment_method": 2,               
        "shipping_billing_match": 1
    })
)

result = response["Body"].read().decode("utf-8")
print(result)  # → something like: [0.842]


[0.8736703664064407]